In [1]:
# 라이브러리 불러오기

from keras.models import load_model
import tensorflow as tf

import matplotlib.pyplot as plt
import os
import shutil
import glob
import cv2

import numpy as np
import pandas as pd

from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.utils import to_categorical
import random

import time

In [2]:
# 저장한 학습시킨 모델 불러오기

model = load_model('./학습함수/CNN_epoch_100_batch_32.h5')

In [50]:
# 경로 지정하기

# 파일이 있는 경로
train_dir = '../dogs-vs-cats/train/train'

# test용 폴더 경로
test_set_dir = os.path.join(train_dir, 'test_set')
test_dog_dir = os.path.join(test_set_dir, 'dog')
test_cat_dir = os.path.join(test_set_dir, 'cat')

# test set 1000장(cat:500/dog:500)
print(f'the number of test set : {len(os.listdir(test_dog_dir)) + len(os.listdir(test_cat_dir))}')

the number of test set : 1000


### 강아지 데이터

In [13]:
dog_test = os.listdir(test_dog_dir)
dog_test_df = pd.DataFrame({
    'f_name':dog_test
})
dog_samples = dog_test_df.shape[0]

In [18]:
dog_test_gen = ImageDataGenerator(rescale=1./255)
dog_test_generator = dog_test_gen.flow_from_dataframe(dog_test_df,
                                             test_dog_dir,
                                             x_col = 'f_name',
                                             y_col=None,
                                             target_size=(150,150),
                                             batch_size=32,
                                             shuffle=False,
                                             class_mode=None)

Found 500 validated image filenames.


In [42]:
predict_dog = model.predict_generator(dog_test_generator, steps=np.ceil(dog_samples/32))

C:\Users\Public\anaconda3\lib\site-packages\keras\engine\training.py:1976: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


In [46]:
predict_dog = pd.DataFrame(predict_dog)
predict_dog.loc[predict_dog[0]>=0.5]

,0
0,0.996181
1,0.996321
2,1.000000
3,0.999702
4,0.992599
...,...
495,0.979219
496,0.727973
497,1.000000
498,1.000000


In [52]:
dog_test_df['predicted'] = predict_dog

categoris = []
j=0
for i in dog_test_df['predicted']:
    if dog_test_df['predicted'][j] < 0.5:
        categoris.append('cat')
        j+=1
    else:
        categoris.append('dog')
        j+=1
        
dog_test_df['category'] = categoris
dog_test_df

,f_name,predicted,category
0,dog.12000.jpg,0.996181,dog
1,dog.12001.jpg,0.996321,dog
2,dog.12002.jpg,1.000000,dog
3,dog.12003.jpg,0.999702,dog
4,dog.12004.jpg,0.992599,dog
...,...,...,...
495,dog.12495.jpg,0.979219,dog
496,dog.12496.jpg,0.727973,dog
497,dog.12497.jpg,1.000000,dog
498,dog.12498.jpg,1.000000,dog


### 고양이 데이터

In [32]:
cat_test = os.listdir(test_cat_dir)
cat_test_df = pd.DataFrame({
    'f_name':cat_test
})
cat_samples = cat_test_df.shape[0]

In [34]:
cat_test_gen = ImageDataGenerator(rescale=1./255)
cat_test_generator = cat_test_gen.flow_from_dataframe(cat_test_df,
                                             test_cat_dir,
                                             x_col = 'f_name',
                                             y_col=None,
                                             target_size=(150,150),
                                             batch_size=32,
                                             shuffle=False,
                                             class_mode=None)

Found 500 validated image filenames.


In [47]:
predict_cat = model.predict_generator(cat_test_generator, steps=np.ceil(cat_samples/32))

C:\Users\Public\anaconda3\lib\site-packages\keras\engine\training.py:1976: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


In [48]:
predict_cat = pd.DataFrame(predict_cat)
predict_cat.loc[predict_cat[0]<0.5]

,0
0,8.548200e-04
1,7.151365e-04
2,1.813504e-02
4,5.789891e-02
5,9.683960e-20
...,...
493,5.775988e-04
494,7.712114e-13
495,2.342796e-01
497,7.883283e-12


In [53]:
cat_test_df['predicted'] = predict_cat

categoris = []
j=0
for i in cat_test_df['predicted']:
    if cat_test_df['predicted'][j] < 0.5:
        categoris.append('cat')
        j+=1
    else:
        categoris.append('dog')
        j+=1
        
cat_test_df['category'] = categoris
cat_test_df

,f_name,predicted,category
0,cat.12000.jpg,8.548200e-04,cat
1,cat.12001.jpg,7.151365e-04,cat
2,cat.12002.jpg,1.813504e-02,cat
3,cat.12003.jpg,7.338394e-01,dog
4,cat.12004.jpg,5.789891e-02,cat
...,...,...,...
495,cat.12495.jpg,2.342796e-01,cat
496,cat.12496.jpg,9.266406e-01,dog
497,cat.12497.jpg,7.883283e-12,cat
498,cat.12498.jpg,8.366362e-01,dog


In [64]:
dogs_error = dog_test_df['category'].loc[dog_test_df['category'] == 'cat'].count()
cats_error = cat_test_df['category'].loc[cat_test_df['category'] == 'dog'].count()

print('dogs_error : ', dogs_error)
print('cats_error : ', cats_error)
print('======================================')
print(f'dogs_error_percent :{dogs_error/500*100: .2f}')
print(f'cats_error_percent : {cats_error/500*100: .2f}')
print(f'all_error_percent : {(dogs_error+cats_error)/1000*100: .2f}')

dogs_error :  21
cats_error :  42
dogs_error_percent : 4.20
cats_error_percent :  8.40
all_error_percent :  6.30
